In [157]:
import pandas as pd
import json
import re
import numpy as np
import csv

#opens file
file = open("sampleLOB.txt")

In [158]:
#stores data from file in array and strips whitespace
data = [line for line in file.readlines() if re.search("[\w].*", line) != None][1:]

In [159]:
organisedData = []
#converts to human readable lines
for line in data:
    for word in line.split(" "):
        if word != '\x00':
            charArray = word.split("\x00")[:-1]
            modifiedWord = ""
            for char in charArray:
                modifiedWord += char
                
            #removes speech marks and commas of time, bid and ask lines
            if "time" in modifiedWord or "bid" in modifiedWord or "ask" in modifiedWord:
                modifiedWord = modifiedWord[1:-2]
            organisedData.append(modifiedWord)

In [160]:
def chunkArrayByWord(ls,chunkWord):   
    chunkedData = []
    chunk = []
    for line in ls:
        if line==chunkWord:
            chunkedData.append(chunk)
            chunk = []
        else:
            chunk.append(line)
    return chunkedData

chunkedData = chunkArrayByWord(organisedData, 'time')[1:]

#remove entries where there are no entries in the order log book
chunkedData = list(filter(lambda timeEntry: len(timeEntry)>3, chunkedData))

evenChunkierData = []
for time in chunkedData:
    bidPos = 1 #it will always be 1
    askPos = time.index('ask')
    evenChunkierData.append([time[0], time[bidPos+1:askPos], time[askPos+1:]])

organisedData = []
for time in evenChunkierData:
    bids = []
    asks = []
    for i, bid in enumerate(time[1]):
        if i % 2 != 0:
            bids.append(time[1][i-1:i+1]) #reconsider
    for i, ask in enumerate(time[2]):
        if i % 2 != 0:
            asks.append(time[2][i-1:i+1]) #reconsider
    organisedData.append([time[0], bids, asks])

In [166]:
# now we aim to flatten the data into tabular data which we will eventually use for the csv file
tabularData =[]
for time in organisedData:
    for bids in time[1]:
        tabularData.append([
            float(time[0][:-1]), #removes comma and converts to float
            'bid',
            int(bids[0][:-1]), #removes comma and converts to int
            int(bids[1]) #converts to int
        ])
    for asks in time[2]:
        tabularData.append([
            float(time[0][:-1]), #removes comma and converts to float
            'ask',
            int(asks[0][:-1]),#removes comma and converts to int
            int(asks[1]) #converts to int
        ])       
np.array(tabularData)[:,3]

array(['3', '3', '3', ..., '4', '2', '5'], dtype='<U32')

In [167]:
#for the time being I am going to save this as csv data
dict = {
    'Time': np.array(tabularData)[:,0],
    'Transaction': np.array(tabularData)[:,1],
    'Value': np.array(tabularData)[:,2],
    'Amount': np.array(tabularData)[:,3],
}  
       
df = pd.DataFrame(dict)
df.to_csv('tabularData.csv') 

In [154]:
#we now need to the start and end time of a bid or ask -> STILL BEING WORKED ON
def createEntryWithStartAndEndTime(tabularData):
    entryOfInterest = tabularData[0]
    indexesOfIntrest = []
    startDate = tabularData[0][0]
    endDate = -1
    for i, entry in enumerate(tabularData):
        if entryOfInterest[-3:] == entry[-3:]:
            indexesOfIntrest.append(i)
            endDate = tabularData[indexesOfIntrest[-1]][0]
    
    for i in indexesOfIntrest: #need to check that there are not gaps in the indexes
        tabularData.pop(i)
    
    return [([startDate, endDate] + entryOfInterest[1:])], tabularData

entry, data = createEntryWithStartAndEndTime(tabularData)
print(entry)
print(list(filter(lambda e: e[-3:] == ['bid', 27, 3], data)))

[[0.496, 2.576, 'bid', 27, 3]]
[[0.528, 'bid', 27, 3], [0.592, 'bid', 27, 3], [0.656, 'bid', 27, 3], [0.72, 'bid', 27, 3], [0.784, 'bid', 27, 3], [0.848, 'bid', 27, 3], [0.912, 'bid', 27, 3], [0.976, 'bid', 27, 3], [1.04, 'bid', 27, 3], [1.104, 'bid', 27, 3], [1.168, 'bid', 27, 3], [1.232, 'bid', 27, 3], [1.296, 'bid', 27, 3], [1.36, 'bid', 27, 3], [1.424, 'bid', 27, 3], [1.488, 'bid', 27, 3], [1.52, 'bid', 27, 3], [1.552, 'bid', 27, 3], [1.584, 'bid', 27, 3], [1.616, 'bid', 27, 3], [1.648, 'bid', 27, 3], [1.68, 'bid', 27, 3], [1.728, 'bid', 27, 3], [1.792, 'bid', 27, 3], [1.856, 'bid', 27, 3], [2.112, 'bid', 27, 3], [2.16, 'bid', 27, 3], [2.176, 'bid', 27, 3], [2.192, 'bid', 27, 3], [2.224, 'bid', 27, 3], [2.256, 'bid', 27, 3], [2.304, 'bid', 27, 3], [2.32, 'bid', 27, 3], [2.368, 'bid', 27, 3], [2.384, 'bid', 27, 3], [2.544, 'bid', 27, 3], [2.56, 'bid', 27, 3]]


In [92]:
#ADITYA'S WORK DO NOT TOUCH
rep = {" ":"", "\"":"", ",":""}

rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))

str = ""
list1 = []
list2=[]

matches = ["[", "]","\n","\t"]


count = 0
for f in file:
    count +=1
    line = f.strip()
    f_line = pattern.sub(lambda m: rep[re.escape(m.group(0))], line)
    if any(x in f_line for x in matches):
        continue
    else:
#         str+=f_line
#         print (str)
        list1.append(f_line)

list1 = list(filter(lambda a: a != {" "}, list1))
print(count)

for i in list1:
    print(i)
    
# print(str)
# list1 = str.split(" ")
# print(list1[0])
# for i in str:
#     if(i!=None and i!=" " and i!="\n" and i!="\t"):
#         list2.append(i)
# print(len(list1))
# print(list1[0])
# print(list2[0])

0


In [3]:
#ADITYA'S WORK DO NOT TOUCH
with open('sampleLOB_50.txt') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        current_line = pattern.sub(lambda m: rep[re.escape(m.group(0))], lines[i].strip())
        print(current_line)

ÿþ[ 
 
      t i m e   
 
     0 . 0  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
     0 . 0 1 6  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
     0 . 0 3 2  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
     0 . 0 4 8  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
 
